In [2]:
#Questions
    #What to do with unbalanced data
    #How to align test and train data from feature engineering

In [3]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [4]:
from fastai.structured import *
from fastai.column_data import *
from sklearn.metrics import roc_auc_score

np.set_printoptions(threshold = 50, edgeitems = 20)

PATH = 'data/'

In [5]:
ls {PATH}

application_test.csv         HomeCredit_columns_description.csv
application_test.csv.zip     installments_payments.csv
application_train.csv        installments_payments.csv.zip
application_train.csv.zip    POS_CASH_balance.csv
bureau_balance.csv           POS_CASH_balance.csv.zip
bureau_balance.csv.zip       previous_application.csv
bureau.csv                   previous_application.csv.zip
bureau.csv.zip               sample_submission.csv
credit_card_balance.csv      sample_submission.csv.zip
credit_card_balance.csv.zip  subm/
df_pref                      tmp/


In [40]:
#data = pd.read_feather('tmp/data')
data = pd.read_csv(PATH + 'application_train.csv')
data.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
#data_test = pd.read_feather('tmp/data_test')
data_test = pd.read_csv(PATH + 'application_test.csv')
data_test.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# nrow = 1000
#application_train = pd.read_csv(path + 'application_train.csv', nrow = 1000)
#bureau_balance = pd.read_csv(path + 'bureau_balance.csv')
#bureau = pd.read_csv(path + 'bureau.csv')
#credit_card_balance = pd.read_csv(path + 'credit_card_balance.csv')
#installments_payments = pd.read_csv(path + 'installments_payments.csv')
#pos_cash_balance = pd.read_csv(path+'POS_CASH_balance.csv')
#previous_application = pd.read_csv(path + 'previous_application.csv')

In [9]:
cat_vars = list(data.select_dtypes('object').columns); len(cat_vars)


16

In [10]:
cat_vars

['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'ORGANIZATION_TYPE',
 'FONDKAPREMONT_MODE',
 'HOUSETYPE_MODE',
 'WALLSMATERIAL_MODE',
 'EMERGENCYSTATE_MODE']

In [11]:
contin_vars = list(data.select_dtypes(np.number).columns); len(contin_vars)

106

In [12]:
#dep = 'TARGET'
#data = data[cat_vars+contin_vars+[dep, 'SK_ID_CURR']].copy()

In [13]:
#data_test[dep] = 0
#data_test = data_test[cat_vars+contin_vars+[dep, 'SK_ID_CURR']].copy()

In [14]:
#for v in cat_vars: data[v] = data[v].astype('category').cat.as_ordered()

In [15]:
#data.select_dtypes('category')

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,WEEKDAY_APPR_PROCESS_START,ORGANIZATION_TYPE,FONDKAPREMONT_MODE,HOUSETYPE_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE
0,Cash loans,M,N,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Laborers,WEDNESDAY,Business Entity Type 3,reg oper account,block of flats,"Stone, brick",No
1,Cash loans,F,N,N,Family,State servant,Higher education,Married,House / apartment,Core staff,MONDAY,School,reg oper account,block of flats,Block,No
2,Revolving loans,M,Y,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Laborers,MONDAY,Government,NaN,NaN,NaN,NaN
3,Cash loans,F,N,Y,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,Laborers,WEDNESDAY,Business Entity Type 3,NaN,NaN,NaN,NaN
4,Cash loans,M,N,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Core staff,THURSDAY,Religion,NaN,NaN,NaN,NaN
5,Cash loans,M,N,Y,"Spouse, partner",State servant,Secondary / secondary special,Married,House / apartment,Laborers,WEDNESDAY,Other,NaN,NaN,NaN,NaN
6,Cash loans,F,Y,Y,Unaccompanied,Commercial associate,Higher education,Married,House / apartment,Accountants,SUNDAY,Business Entity Type 3,NaN,NaN,NaN,NaN
7,Cash loans,M,Y,Y,Unaccompanied,State servant,Higher education,Married,House / apartment,Managers,MONDAY,Other,NaN,NaN,NaN,NaN
8,Cash loans,F,N,Y,Children,Pensioner,Secondary / secondary special,Married,House / apartment,NaN,WEDNESDAY,XNA,NaN,NaN,NaN,NaN
9,Revolving loans,M,N,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Laborers,THURSDAY,Electricity,NaN,NaN,NaN,NaN


In [24]:
#data_test.dtypes

NAME_CONTRACT_TYPE            category
CODE_GENDER                   category
FLAG_OWN_CAR                  category
FLAG_OWN_REALTY               category
NAME_TYPE_SUITE               category
NAME_INCOME_TYPE              category
NAME_EDUCATION_TYPE           category
NAME_FAMILY_STATUS            category
NAME_HOUSING_TYPE             category
OCCUPATION_TYPE               category
WEEKDAY_APPR_PROCESS_START    category
ORGANIZATION_TYPE             category
FONDKAPREMONT_MODE            category
HOUSETYPE_MODE                category
WALLSMATERIAL_MODE            category
EMERGENCYSTATE_MODE           category
SK_ID_CURR                       int64
TARGET                           int64
CNT_CHILDREN                     int64
AMT_INCOME_TOTAL               float64
AMT_CREDIT                     float64
AMT_ANNUITY                    float64
AMT_GOODS_PRICE                float64
REGION_POPULATION_RELATIVE     float64
DAYS_BIRTH                       int64
DAYS_EMPLOYED            

In [42]:
train_cats(data)

In [43]:
apply_cats(data_test, data)

In [44]:
data.shape, data_test.shape

((307511, 122), (48744, 121))

# Deep Learning

Merics

In [7]:
def rmse(x,y):
    return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [roc_auc_score(y_train, m.predict(X_train)), roc_auc_score(y_valid, m.predict(X_valid)),
          m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m,'oob_score_'): res.append(m.oob_score_)
    print(res)

Run on a sample

In [48]:
n = len(data)
idxs = get_cv_idxs(n, val_pct = 50000/n)
data_samp = data.iloc[idxs].set_index('SK_ID_CURR')
samp_size = len(data_samp); samp_size

50000

Run on full set

In [ ]:
sample_size = len(data)
data_samp = data.set_index('SK_ID_CURR')
data_samp.shape

In [51]:
data_samp.head()

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
384575,0,Cash loans,M,Y,N,2,207000.0,465457.5,52641.0,418500.0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,1.0
214010,0,Cash loans,F,Y,Y,0,247500.0,1281712.5,48946.5,1179000.0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,3.0
142232,0,Cash loans,F,Y,N,0,202500.0,495000.0,39109.5,495000.0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,3.0
389171,0,Cash loans,F,N,Y,0,247500.0,254700.0,24939.0,225000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
283617,0,Cash loans,M,N,Y,0,112500.0,308133.0,15862.5,234000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,4.0


ModelData object

In [52]:
df, y, nas, mapper = proc_df(data_samp, 'TARGET', do_scale=True)

In [62]:
data_test['TARGET'] = 0 #??

In [63]:
df_test, _, nas, mapper = proc_df(data_test, 'TARGET', do_scale=True, 
                    skip_flds=['SK_ID_CURR'], mapper=mapper, na_dict=nas)

In [60]:
#validation index
train_ratio = 0.75
train_size = int(samp_size * train_ratio); train_size
val_idx = list(range(train_size, len(df))); len(val_idx)

12500

In [65]:
md = ColumnarModelData.from_data_frame(PATH, val_idx, df, yl.astype(np.float32), 
                                       cat_flds = cat_vars, bs=128, test_df=df_test )

NameError: name 'yl' is not defined